In [2]:
import argparse
import os
import random
import shutil
import time
import warnings

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim
import torch.utils.data
import torch.nn.functional as F


from torch.utils import data
import random
import numpy as np
from itertools import product
import argparse

from train_f import *
from Dataset import Dataset
from Models import *
from args import args


In [11]:
data_range = 1024
pos=list(np.arange(0,data_range,32))
ranges=list(product(pos,repeat=3))

train_data = ranges[:int(np.round(len(ranges)*0.6))]
val_data=ranges[int(np.round(len(ranges)*0.6)):int(np.round(len(ranges)*0.8))]
test_data = ranges[int(np.round(len(ranges)*0.8)):]

In [12]:
train_data

[(0, 0, 0),
 (0, 0, 32),
 (0, 0, 64),
 (0, 0, 96),
 (0, 0, 128),
 (0, 0, 160),
 (0, 0, 192),
 (0, 0, 224),
 (0, 0, 256),
 (0, 0, 288),
 (0, 0, 320),
 (0, 0, 352),
 (0, 0, 384),
 (0, 0, 416),
 (0, 0, 448),
 (0, 0, 480),
 (0, 0, 512),
 (0, 0, 544),
 (0, 0, 576),
 (0, 0, 608),
 (0, 0, 640),
 (0, 0, 672),
 (0, 0, 704),
 (0, 0, 736),
 (0, 0, 768),
 (0, 0, 800),
 (0, 0, 832),
 (0, 0, 864),
 (0, 0, 896),
 (0, 0, 928),
 (0, 0, 960),
 (0, 0, 992),
 (0, 32, 0),
 (0, 32, 32),
 (0, 32, 64),
 (0, 32, 96),
 (0, 32, 128),
 (0, 32, 160),
 (0, 32, 192),
 (0, 32, 224),
 (0, 32, 256),
 (0, 32, 288),
 (0, 32, 320),
 (0, 32, 352),
 (0, 32, 384),
 (0, 32, 416),
 (0, 32, 448),
 (0, 32, 480),
 (0, 32, 512),
 (0, 32, 544),
 (0, 32, 576),
 (0, 32, 608),
 (0, 32, 640),
 (0, 32, 672),
 (0, 32, 704),
 (0, 32, 736),
 (0, 32, 768),
 (0, 32, 800),
 (0, 32, 832),
 (0, 32, 864),
 (0, 32, 896),
 (0, 32, 928),
 (0, 32, 960),
 (0, 32, 992),
 (0, 64, 0),
 (0, 64, 32),
 (0, 64, 64),
 (0, 64, 96),
 (0, 64, 128),
 (0, 64, 160

In [5]:
pos=list(np.arange(0,data_range,32))


In [13]:
# whole box, volumn = 1024^3 = 10.7 * 10^8
# train, volumn = 6.7 * 10^8
# val idx(0-416, 0-416, 0-992), volumn = 448 * 448 * 1024 = 2.1 * 10^8
# test idx(448-992, 448-992, 448-992), volumn =  576^3 = 1.9 * 10^8
train_data, val_data, test_data = [],[],[]

for i in range(0,data_range,32):
    for j in range(0,data_range,32):
        for k in range(0,data_range,32):
            idx = (i,j,k)
            if i <=416 and j<= 416:
                val_data.append(idx)
            elif i>=484 and j>= 448 and k>= 448:
                test_data.append(idx)
            else:
                train_data.append(idx)

In [14]:
print(test_data)

[(512, 448, 448), (512, 448, 480), (512, 448, 512), (512, 448, 544), (512, 448, 576), (512, 448, 608), (512, 448, 640), (512, 448, 672), (512, 448, 704), (512, 448, 736), (512, 448, 768), (512, 448, 800), (512, 448, 832), (512, 448, 864), (512, 448, 896), (512, 448, 928), (512, 448, 960), (512, 448, 992), (512, 480, 448), (512, 480, 480), (512, 480, 512), (512, 480, 544), (512, 480, 576), (512, 480, 608), (512, 480, 640), (512, 480, 672), (512, 480, 704), (512, 480, 736), (512, 480, 768), (512, 480, 800), (512, 480, 832), (512, 480, 864), (512, 480, 896), (512, 480, 928), (512, 480, 960), (512, 480, 992), (512, 512, 448), (512, 512, 480), (512, 512, 512), (512, 512, 544), (512, 512, 576), (512, 512, 608), (512, 512, 640), (512, 512, 672), (512, 512, 704), (512, 512, 736), (512, 512, 768), (512, 512, 800), (512, 512, 832), (512, 512, 864), (512, 512, 896), (512, 512, 928), (512, 512, 960), (512, 512, 992), (512, 544, 448), (512, 544, 480), (512, 544, 512), (512, 544, 544), (512, 544, 57

In [16]:
print(val_data)

[(0, 0, 0), (0, 0, 32), (0, 0, 64), (0, 0, 96), (0, 0, 128), (0, 0, 160), (0, 0, 192), (0, 0, 224), (0, 0, 256), (0, 0, 288), (0, 0, 320), (0, 0, 352), (0, 0, 384), (0, 0, 416), (0, 0, 448), (0, 0, 480), (0, 0, 512), (0, 0, 544), (0, 0, 576), (0, 0, 608), (0, 0, 640), (0, 0, 672), (0, 0, 704), (0, 0, 736), (0, 0, 768), (0, 0, 800), (0, 0, 832), (0, 0, 864), (0, 0, 896), (0, 0, 928), (0, 0, 960), (0, 0, 992), (0, 32, 0), (0, 32, 32), (0, 32, 64), (0, 32, 96), (0, 32, 128), (0, 32, 160), (0, 32, 192), (0, 32, 224), (0, 32, 256), (0, 32, 288), (0, 32, 320), (0, 32, 352), (0, 32, 384), (0, 32, 416), (0, 32, 448), (0, 32, 480), (0, 32, 512), (0, 32, 544), (0, 32, 576), (0, 32, 608), (0, 32, 640), (0, 32, 672), (0, 32, 704), (0, 32, 736), (0, 32, 768), (0, 32, 800), (0, 32, 832), (0, 32, 864), (0, 32, 896), (0, 32, 928), (0, 32, 960), (0, 32, 992), (0, 64, 0), (0, 64, 32), (0, 64, 64), (0, 64, 96), (0, 64, 128), (0, 64, 160), (0, 64, 192), (0, 64, 224), (0, 64, 256), (0, 64, 288), (0, 64, 32

In [17]:
print(train_data)

[(0, 448, 0), (0, 448, 32), (0, 448, 64), (0, 448, 96), (0, 448, 128), (0, 448, 160), (0, 448, 192), (0, 448, 224), (0, 448, 256), (0, 448, 288), (0, 448, 320), (0, 448, 352), (0, 448, 384), (0, 448, 416), (0, 448, 448), (0, 448, 480), (0, 448, 512), (0, 448, 544), (0, 448, 576), (0, 448, 608), (0, 448, 640), (0, 448, 672), (0, 448, 704), (0, 448, 736), (0, 448, 768), (0, 448, 800), (0, 448, 832), (0, 448, 864), (0, 448, 896), (0, 448, 928), (0, 448, 960), (0, 448, 992), (0, 480, 0), (0, 480, 32), (0, 480, 64), (0, 480, 96), (0, 480, 128), (0, 480, 160), (0, 480, 192), (0, 480, 224), (0, 480, 256), (0, 480, 288), (0, 480, 320), (0, 480, 352), (0, 480, 384), (0, 480, 416), (0, 480, 448), (0, 480, 480), (0, 480, 512), (0, 480, 544), (0, 480, 576), (0, 480, 608), (0, 480, 640), (0, 480, 672), (0, 480, 704), (0, 480, 736), (0, 480, 768), (0, 480, 800), (0, 480, 832), (0, 480, 864), (0, 480, 896), (0, 480, 928), (0, 480, 960), (0, 480, 992), (0, 512, 0), (0, 512, 32), (0, 512, 64), (0, 512,